In [1]:
pip install kaggle
! pip install -q kaggle
import os
if not os.path.isfile(os.path.expanduser('~/.kaggle/kaggle.json')):
  from google.colab import files
  print("Upload kaggle.json here")
  files.upload()

if not os.path.isfile('IMDB Dataset.csv'):
  !mkdir ~/.kaggle
  !mv ./kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  dataset_name = 'mohamedhanyyy/chest-ctscan-images'
  zip_name = dataset_name.split('/')[-1]

  !kaggle datasets download -d {dataset_name}
  !unzip -q ./{zip_name}.zip -d .

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3


import warnings
warnings.filterwarnings("ignore")
# File Directory for both the train and test
train_path = "C:\Users\Admin\Downloads\CODE\CODE\DATA\Data\train"
val_path = "C:\Users\Admin\Downloads\CODE\CODE\DATA\Data\valid"
test_path = "C:\Users\Admin\Downloads\CODE\CODE\DATA\Data\test"
# Define function to count number of images per class using a dictionary
def GetDatasetSize(path):
    num_of_image = {}
    for folder in os.listdir(path):
        # count files in the folder
        num_of_image[folder] = len(os.listdir(os.path.join(path, folder)));
    return num_of_image;

# Get the number of images per class in each set (train, validation and test)
train_set = GetDatasetSize(train_path)
val_set = GetDatasetSize(val_path)
test_set = GetDatasetSize(test_path)
print(train_set,"\n\n",val_set,"\n\n",test_set)
# Labels for each classs
labels = ['squamous.cell.carcinoma', 'normal', 'adenocarcinoma', 'large.cell.carcinoma']

# Create lists from previous dictionaries storing the count of images per category
train_list = list(train_set.values())
val_list = list(val_set.values())
test_list = list(test_set.values())

# Labels location and bars widht
x = np.arange(len(labels))
width = 0.25

# Create plot and 3 sets of bars (train, val, test)
fig, ax = plt.subplots()
rects1 = ax.bar(x - width, train_list, width, label='Train')
rects2 = ax.bar(x, val_list, width, label='Val')
rects3 = ax.bar(x + width, test_list, width, label='Test')

# Add labels, title, legend, count values...
ax.set_ylabel('Images Count')
ax.set_title('Dataset')
ax.set_xticks(x, labels)
plt.xticks(rotation=15)
ax.legend()
ax.bar_label(rects1)
ax.bar_label(rects2)
ax.bar_label(rects3)

# Optimized layout and displaying plot
fig.tight_layout()
plt.show()
# Image data generator with specified augmentation configurations (mostly geometric transformations)
train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest',
                                  zoom_range=0.2,
                                  shear_range = 0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  rotation_range=0.4)

# Using data generator to create augmented data from image files in train_path directory
train_data = train_datagen.flow_from_directory(train_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')

# Dicctionary with class names to their respective indices in the generated data
train_data.class_indices
val_datagen = ImageDataGenerator(rescale = 1.0/255.0)
val_data = val_datagen.flow_from_directory(val_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')
val_data.class_indices
{'adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib': 0,
 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa': 1,
 'normal': 2,
 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa': 3}
test_datagen = ImageDataGenerator(rescale = 1.0/255.0)
test_data = test_datagen.flow_from_directory(test_path,
                                                   batch_size = 5,
                                                   target_size = (350,350),
                                                   class_mode = 'categorical')
test_data.class_indices
{'adenocarcinoma': 0,
 'large.cell.carcinoma': 1,
 'normal': 2,
 'squamous.cell.carcinoma': 3}
# Function to plot sample images with labels
def plot_sample_images(images, labels, class_indices):
    class_labels = list(class_indices.keys())
    fig, axs = plt.subplots(1, len(images), figsize=(15, 5))
    fig.subplots_adjust(wspace=0.5)  # Adjust the width space between subplots

    for i in range(len(images)):
        axs[i].imshow(images[i])
        axs[i].set_title("Label: {}".format(class_labels[np.argmax(labels[i])]), fontsize=10)  # Adjust fontsize as needed
        axs[i].axis('off')

    plt.show()

# Sample and plotting of 3 images with their respective labels from train_data
sample_images, sample_labels = next(train_data)
plot_sample_images(sample_images[:3], sample_labels[:3], train_data.class_indices)
base_model = EfficientNetB0(input_shape = (350, 350, 3),
                         include_top = False,
                         weights = 'imagenet')
mc = ModelCheckpoint(
    filepath="./ct_effnet_best_model.hdf5",
    monitor= 'val_accuracy',
    verbose= 1,
    save_best_only= True,
    mode = 'auto'
    );

call_back = [ mc];

tensorboard = TensorBoard(log_dir = 'logs')
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

#We also add a earlystop for prevent the computer from wasting time if it's not making progress.


early_stopping = EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)
#Customize our base model
EffNetmodel = base_model.output
EffNetmodel = tf.keras.layers.GlobalAveragePooling2D()(EffNetmodel)
# to provide overfitting problem
EffNetmodel = tf.keras.layers.Dropout(rate=0.5)(EffNetmodel)

#Finally, we add a layer with 4 'neurons' that will help us classify things into different categories.
EffNetmodel = tf.keras.layers.Dense(4,activation='softmax')(EffNetmodel)
#We put together the original EfficientNetB0 and our new custom parts
EffNetmodel = tf.keras.models.Model(inputs=base_model.input, outputs = EffNetmodel)

#Compile up the rules and tools for training your neural network.
#The optimizer determines how the model should update its internal parameters, the loss function quantifies how well the model is doing, and the metrics provide additional measures to track the model's performance.
EffNetmodel.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])
%time
EffNetB0 = EffNetmodel.fit(
    train_data,
    steps_per_epoch = train_data.samples//train_data.batch_size,
    epochs = 15,
    validation_data = val_data,
    validation_steps = val_data.samples//val_data.batch_size,
    callbacks = [tensorboard, mc, reduce_lr, early_stopping]
    )
from tensorflow.keras.models import load_model



# Assuming you have trained and named your model 'model'
# model = your_model_training_function()

# Save the model
EffNetB0.save("lungcancer11.h5")

# Optionally, if you want to load the model later:
# loaded_model = load_model("lungcancer.h5")
from tensorflow.keras.models import load_model

# Load the saved model
saved_model_path = "/content/ct_effnet_best_model.hdf5"
model = load_model(saved_model_path)

# Load and preprocess the new image data using the same data generator
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the new image data
img_path = '/content/Data/test/adenocarcinoma/000113 (7).png'  # Update with the path to your new image
img = image.load_img(img_path, target_size=(350, 350))  # Assuming your model expects input size of 350x350
img_array = image.img_to_array(img)
img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to (1, height, width, channels)

# Preprocess the image using the same data generator
train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   horizontal_flip=True,
                                   fill_mode='nearest',
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=0.4)

# No need for augmentation, as we're dealing with a single image
img_generator = train_datagen.flow(img_array, batch_size=1)

# Now you can use the model to make predictions
predictions = model.predict(img_generator)

# Print the predictions
print(predictions)

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
saved_model_path = "/content/ct_effnet_best_model.hdf5"
model = load_model(saved_model_path)

# Load and preprocess the new image data
img_path = '/content/Data/train/adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib/000009 (3).png'  # Update with the path to your new image
img = image.load_img(img_path, target_size=(350, 350))  # Assuming your model expects input size of 350x350
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Rescale to [0, 1] range (assuming you rescaled images in the ImageDataGenerator)

# Now you can use the model to make predictions
predictions = model.predict(img_array)

# Get the class indices
class_indices = train_data.class_indices

# Invert the dictionary to map indices to class names
inverse_class_indices = dict((v, k) for k, v in class_indices.items())

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Get the predicted class name
predicted_class_name = inverse_class_indices[predicted_class_index]

# Print the predicted class name
print("Predicted class:", predicted_class_name)

pip install pillow
from keras.preprocessing import image
import numpy as np
from PIL import Image
from google.colab import files

# Load and preprocess the image
def load_and_preprocess_image():
    # Prompt the user to upload a file
    uploaded = files.upload()

    # Get the path of the uploaded file
    for filename in uploaded.keys():
        image_path = filename

    # Load and preprocess the new image
    img = Image.open(image_path)
    img = img.resize((350, 350))  # Adjust the target size to match your model's input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize pixel values to the range [0, 1]

    return img_array

# Load your own model
model = load_model("/content/ct_effnet_best_model.hdf5")  # Replace "your_model.h5" with the path to your model file

# Example usage:
new_image = load_and_preprocess_image()

# Assuming your model is loaded and named 'model'
predictions = model.predict(new_image)

# Process the predictions as needed for your application
predicted_class_index = np.argmax(predictions)
print("Predicted Class Index:", predicted_class_index)
